### Задание. Прочитать из файла матрицу смежности и с помощью алгоритма Дейкстры определить кратчайшее расстояние от заданной вершины до всех остальных.

#### Чтобы визуализировать граф в виде кружочков и стрелочек, установим библиотеку pyvis

### Выполнение:

In [4]:
!pip install pyvis

In [5]:
# Считывание матрицы вида [[1,2], [4, 5]] 
with open("lr3_1.txt", 'r') as file:
    data = file.read()

# Преобразование элементов в матрицу
elements = data.strip('[]').split('],[')
matrix = list(list(map(int, item.split(','))) for item in elements)

print(matrix)

# Этот вариант генерирует примитивную матрицу большого размера. Она пригодится для оценки сложности алгоритма
# matrix = [[i * j for j in range(10000)] for i in range(10000)]

[[4, 9, 0, 2, 0, 6], [4, 6, 7, 7, 1, 1], [3, 2, 9, 0, 2, 1], [0, 0, 2, 5, 4, 9], [3, 6, 8, 4, 6, 4], [9, 4, 9, 2, 3, 0]]


In [6]:
from pyvis.network import Network

graph = Network()

# Добавление элементов матрицы в объект графа как вершины
for i in range(len(matrix)):
    graph.add_node(i, label=str(i), shape='circle', font={'size': 20})

# Добавление стрелок в ориентированный граф
for i in range(len(matrix)):
    for j in range(len(matrix[i])):
        if matrix[i][j] != 0:
            graph.add_edge(i, j, label=str(matrix[i][j]), arrows='to')

# Сохранение графа в .html и запуск 
graph.show_buttons(filter_=['nodes', 'edges', 'physics'])
graph.write_html('graph.html')
graph.show('graph.html', notebook=False)

graph.html


### Алгоритм Дейкстры в первом шаге берет вершину, от котрой мы хотим посчитать расстояния до остальных. На каждой итерации алгоритм считает веса до вершин, которые доступны из данной. Если вес меньше, чем на предыдущих итерациях, то вес обновляется. Алгоритм проходит по каждой вершине, до каких можно добраться из первоначальных. 

In [7]:
# Вычисляем вершины, до которых можно добраться из данной
def get_link(v, ms):
    for i, weight in enumerate(ms[v]):
        if weight > 0:
            yield i

# Берем вершину с наименьшим весом. С нее начинаем следующую итерацию
def arg_min(T, S):
    amin = -1
    mmin = float('inf')
    for i, k in enumerate(T):
        if k < mmin and not S[i]:
            mmin = k
            amin = i

    return amin

# Проход по вершинам и обновление весов
def dijkstra(ms, v):
    N = len(ms)
    T = [float('inf')] * N
    S = [False] * N
    T[v] = 0
    
    while v != -1:
        v = arg_min(T, S)
        S[v] = True
        for j in get_link(v, ms):
            if not S[j]:
                w = T[v] + ms[v][j]
                if w < T[j]:
                    T[j] = w
    
    return T

In [8]:
# Подсчет времени выполнения для разного числа вершин
import time

start_time = time.time()

res = dijkstra(matrix, 5)

end_time = time.time()

print(res, end_time - start_time, 's')

[6, 4, 4, 2, 3, 0] 0.0 s


### С помощью библиотеки tabulate изобразим результаты в виде таблицы

In [9]:
!pip install tabulate

In [10]:
from tabulate import tabulate

data = [
    ["100", "около 10.000", "0.002 s"],
    ["1000", "около 1.000.000", "0.147s"],
    ["10000", 'около 100.000.000', "15.062 s"]
]

headers = ["Количество узлов", "Количество ребер", "Время выполнения"]

table = tabulate(data, headers=headers, tablefmt="grid")
print(table)

+--------------------+--------------------+--------------------+
|   Количество узлов | Количество ребер   | Время выполнения   |
+====================+====================+====================+
|                100 | около 10.000       | 0.002 s            |
+--------------------+--------------------+--------------------+
|               1000 | около 1.000.000    | 0.147s             |
+--------------------+--------------------+--------------------+
|              10000 | около 100.000.000  | 15.062 s           |
+--------------------+--------------------+--------------------+


### Вывод

Алгоритм Дейкстры, который я реализовал в данной лабораторной работе,
работает по сложности O(n^2), где n - количестов вершин. Так как для каждой из n вершин считается расстояние до остальных n-1 вершин
Данная зависимость прослеживается по небольшой таблице, составленной по результатам эксперимента.
При увеличении количества вершин в 10 раз, время работы увеличивается примерно в 100 раз.